# Prompt Templates

Es una estrcutura que toam un diccionario como entrada, donde cada clave  representa  una varaible que debe rellnarse en el template.

Se usan para convertir la entrada del usuario en un instruccion clara para el modelo

Puede ser convertido en una cadena de texto o lista de mensajes

In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

String Prompt Template: Es el tipo más sencillo, donde se toma una entrada y se convierte en una cadena de texto. Se usa para crear prompts básicos.

    Ejemplo: Un template podría ser "Dime un chiste sobre {topic}". Luego, se pasa un diccionario con {"topic": "gatos"}, y la salida sería "Dime un chiste sobre gatos".



In [ ]:
# Como siemrpe si quieres instalarlo en un entorno especifico o en tu vs code, es tu eleccion

#pip install -qU langchain-openai langchain

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-4o")

In [27]:
from langchain_core.prompts import PromptTemplate

# Recuerda que el modelo procesa cualquier idioma " ver esto con otro modelos mas raros"
prompt_template = PromptTemplate.from_template("Dime un chiste {topic}")

print(prompt_template.invoke({"topic":"gatos"}))

text='Dime un chiste gatos'


Chat Prompt Template: Se utiliza para crear una conversación con diferentes roles, como el sistema, el usuario, o un asistente. Permite generar listas de mensajes estructuradas.

In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", " Eres un asistente util"),
    ("user", " dime un chiste {topic}")
])

print(prompt_template.invoke({"topic":"gatos"}))


messages=[SystemMessage(content=' Eres un asistente util', additional_kwargs={}, response_metadata={}), HumanMessage(content=' dime un chiste gatos', additional_kwargs={}, response_metadata={})]


Message Placeholder: Un lugar reservado en el template donde se puede insertar una lista de mensajes. Esto permite añadir múltiples mensajes en un punto específico del template, ideal para diálogos flexibles

Invoke: Método que permite pasar el diccionario de entrada al template y generar la salida final. Esto reemplaza las variables del template con los valores proporcionados. 

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", " Eres un asistente util"),
    MessagesPlaceholder("msgs")

    # Si quisieramos añadir mas informacion u otro rol, tambien lo podemos hacer
])

print(prompt_template.invoke( {"msgs": [ HumanMessage(content= "HI"), HumanMessage(content= "Adios")]} ))



messages=[SystemMessage(content=' Eres un asistente util', additional_kwargs={}, response_metadata={}), HumanMessage(content='HI', additional_kwargs={}, response_metadata={}), HumanMessage(content='Adios', additional_kwargs={}, response_metadata={})]


Los prompt templates nos ayudan a dar instruccion al modelo y mejorar la calidad de la respuesta.

# Few Shot prompting

Es una tecnica que utiliza ejemplos de como debe comportarse le modelo dentro de mismo prompt, permitiendo que el modelo aprenda de ejemplo que nosotros mismos vamos a ir construyendo. ( mejora la precision y relevancia de las repuestas del modelo)

Se implementa:

    Usando ejemplos fijos ( donde usa en todos los casos los mismo ejemplos)
    Usando ejemplos dinamicos ( Seleccionan dinamicamente basandose en la similitud semantica entre la entrada de usario y los ejemplos que se han venido almacenando)

Aca = pueden consultarse ejemplos https://python.langchain.com/docs/how_to/few_shot_examples_chat/

In [30]:

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [ ]:

model.invoke('Cuanto es : 2 🦜 9').content
#Debemos darle ejemplos para que el modelo sepa que es lo que esperamos

In [32]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [{'input': '2 🦜 2', 'output': '4'},
            {'input': '2 🦜 3', 'output': '5'}]

In [33]:
example_prompt = ChatPromptTemplate(
    [('human', '{input}'),
     ('ai', '{output}')]
)

In [34]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

In [35]:
print(few_shot_prompt.invoke({}).to_messages)

<bound method ChatPromptValue.to_messages of ChatPromptValue(messages=[HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})])>


Prompt Template: Pasa ejemplos al modelo. Cada ejemplo tiene una entrada (input) y una salida (output) esperada, generalmente estructurados en formato de diccionario. 

In [36]:
main_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " eres un mago de las matematicas"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

Chains: Permite integrar múltiples pasos en una cadena, combinando el modelo con ejemplos y roles para crear interacciones más complejas. 

In [37]:
# Como utlizar las cadenas

chain =  main_prompt | model

In [ ]:
chain.invoke({"input" : "Cuanto es 2  🦜 9"}).content

# LCEL

LangChain Expression Language, permiten secuencias de pasos llamadas a un modelo que puede ser usado en operaciones complejas o simple ( Facilita la cracion de cadenas con soprte para streaming, ejecucion paralela, asincronia)

Mas info consulta = https://python.langchain.com/docs/how_to/output_parser_structured/#lcel

In [39]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "Traduce lo siguiente al {language}: "),
    ('human', '{text}')
])

In [40]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt_template | model | parser


In [ ]:
response = chain.invoke({'language': 'Italian', 'text': 'hello'})

In [ ]:
print(response)

# Ejercicio: Chat Memory

In [43]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [44]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat_history = []
if not chat_history:
  system_message = SystemMessage(content='Eres un asistente útil')
  chat_history.append(system_message)

In [ ]:
query = input('Haz una pregunta: ')
chat_history.append(HumanMessage(content=query))

response = model.invoke(chat_history).content
chat_history.append(AIMessage(content=response))

In [ ]:
for message in chat_history:
  print(message)

content='Eres un asistente útil'
content='hola'
content='¡Hola! ¿En qué puedo ayudarte hoy?'
content='cual es tu nombre'
content='Soy un asistente virtual desarrollado por OpenAI, y no tengo un nombre propio. Pero estoy aquí para ayudarte con lo que necesites. ¿En qué puedo asistirte hoy?'
content='adios'
content='¡Adiós! Si necesitas ayuda en el futuro, no dudes en volver. ¡Que tengas un buen día!'


In [ ]:
chat_history

# Integración de Cadenas en Proyectos : Runnable, OutputParser y Streaming

Cosulte el enlace para mas info https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html

In [ ]:
#!pip install langchain

In [47]:
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: x +1 ) | RunnableLambda(lambda x: x * 2 )

In [48]:
sequence.invoke(10)

22

In [49]:
sequence = RunnableLambda(lambda x: x +1 ) | {
    'index_1' : RunnableLambda(lambda x: x * 2 ),
    'index_2' : RunnableLambda(lambda x: x * 5 )
}

In [50]:
sequence.invoke(10)

{'index_1': 22, 'index_2': 55}

In [ ]:
#!pip install -qU langchain langchain_openai

In [51]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [52]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [53]:
model = ChatOpenAI()

In [54]:
joke_query = 'Tell me a joke'

In [55]:
parser = JsonOutputParser()

In [56]:
prompt = PromptTemplate(
    template = 'Answer the user query. \n{format_instructions}\n{query}',
    input_variables = ['query'],
    partial_variables = { 'format_instructions': parser.get_format_instructions()}
)

In [57]:
chain = prompt | model | parser

In [ ]:
response = chain.invoke({"query": joke_query})

In [ ]:
response

In [ ]:
import time
for s in chain.stream({"query": joke_query}):
  print(s)
  time.sleep(0.3)

In [ ]:
chunks = []
async for chunk in model.astream(joke_query):
  chunks.append(chunk)
  print(chunk.content, end='', flush=True)
  time.sleep(0.3)

# Chat Memory

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
#!pip install langchain_openai langchain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
model.invoke([HumanMessage(content='Hi! my name is bob')]).content

In [ ]:
model.invoke([HumanMessage(content='What is my name?')]).content

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
).content

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {'configurable': {'session_id': 'abc2'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Bob')],
     config=config
)

In [ ]:
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)

In [ ]:
response.content

In [ ]:
store

In [ ]:
config = {'configurable': {'session_id': 'abc3'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)
response

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Carli')],
     config=config
)
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)
response

In [ ]:
store

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 'You are a helpful assistant. Answer all question to the best of your abity.'
        ),
        MessagesPlaceholder(variable_name='messages')

    ]
)

In [ ]:
chain  = prompt | model

In [ ]:
response = chain.invoke(
    {
        'messages': [HumanMessage(content='Hi! I am Bob')]
    }
)
response.content


In [ ]:
response = chain.invoke(
    {
        'messages': [HumanMessage(content='What is your name?')]
    }
)
response.content

In [ ]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config = { 'configurable': {'session_id': 'abc5'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is your name?')],
    config=config
)
response.content

'I am a helpful assistant here to assist you with any questions or tasks you have. You can call me Assistant. How can I assist you today?'

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Bob')],
    config=config
)
response.content

'Hello Bob! How can I assist you today?'